# Imports

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from itertools import product

In [ ]:
from icesea2.experiments import gen_lhs_from_dict, norm_to_values, topography_to_icesheet_nc, gen_run_configs
from icesea2.margins import shp_series_to_icesheet
from icesea2.shearstress import make_shear_stress_map
from icesea2.models import IcesheetModel

# Run Configuration

In [ ]:
# grid
dx = 5000
dy = 5000
x_min = -1265453.0
x_max = 4159547.0
y_min = -4722734.8
y_max = 1352265.2

grid_crs = '+ellps=WGS84 +proj=laea +lon_0=0.0 +lat_0=90 +x_0=0.0 +y_0=0.0 +no_defs'

shear_stress_map_name = "ely_victor_eurasia"
shear_stress_map_file = "/nfs/annie/cm15ogp/phd/data/shear_stress/eurasia/regions.shp"

shear_stress_gauss_blur = 1
shear_stress_base_value = 5000

times = [-140000]

shear_stress_combine_pattern = "((g_sediment | (g_pgm_ice_streams * p_ice_stream)) | p_hybrid_ice_stream) | p_cold_ice"
shear_stress_sediment_layers = ["sediment", "pgm_ice_streams"]
shear_stress_process_layers = ["ice_stream", "cold_ice", "hybrid_ice_stream"]
shear_stress_parameters_var_ranges = {
    "g_pgm_ice_streams_ice_stream": [5000,20000],
    "g_sediment_marine_sediment": [10000,30000],
    "g_sediment_onshore_sediment": [30000,100000],
    "g_sediment_bedrock": [100000,150000],
    "p_ice_stream_interior_dist": [0,1000],
    "p_cold_ice_shear_stress": [120000,200000],
    "p_cold_ice_interior_dist": [1000,300],
    "p_hybrid_ice_stream_shear_stress": [5000,100000],
    "p_hybrid_ice_stream_dist": [0,600],
}

margins_name = "batchelor_mis6"
margins_dir = "/nfs/annie/cm15ogp/phd/data/ice_margins/batchelor_quickfix"
run_dir = "/nfs/annie/cm15ogp/phd/projects/2020-12-01_icesheet_calibration/experiments/shearstress_lhs_process/batchelor/"

t_var_topo = False
e_var_topo = False
e_var_ss = True
t_var_ss = True
e_var_margins = False

topography_name = "rtopo2_laea_5km_eurasia"
topography_nc = "/nfs/annie/cm15ogp/phd/data/topography/rtopo2/rtopo2_laea_5km_eurasia.nc"

sample_type = "lhs"
n_samples = 200
sample_norm_path = "batchelor/lhs_norm.csv"
sample_values_path = "batchelor/lhs_values.csv"

icesheet_model_binary = "/nfs/annie/cm15ogp/code/icesheet/binaries/icesheet"
num_run_cores = 24


In [ ]:
# corners
shear_stress_parameters_var_ranges = {
    "g_lgm_ice_streams_ice_stream": [5000,20000],
    "g_sediment_marine_sediment": [10000,30000],
    "g_sediment_onshore_sediment": [30000,100000],
    "g_sediment_bedrock": [100000,150000],
    "p_ice_stream_interior_dist": [0,1000],
    "p_cold_ice_shear_stress": [120000,200000],
    "p_cold_ice_interior_dist": [1000,300]
}
shear_stress_parameters_const = {
    "m_base_map_name": shear_stress_map_name,
    "m_base_map_file": shear_stress_map_file,
    "m_gauss_blur": 1,
    "m_base_value": 5000,
    "m_combine_pattern": "(g_sediment | (g_lgm_ice_streams * p_ice_stream)) | p_cold_ice",
    "g_layers": ["sediment", "lgm_ice_streams"],
    "p_processes": ["ice_stream", "cold_ice"],
}
times = [-22000,-20000,-18000,-16000]
margins_name = "smooth_v2_topoclip_mod_-1km"
margins_dir = "/nfs/annie/cm15ogp/phd/data/ice_margins/ice6g/smooth_v2_topoclip_mod_-1km"
run_dir = "/nfs/annie/cm15ogp/phd/paper1/paper1_data/icesheet_runs/ice6g/corners/"

t_var_topo = True
e_var_topo = False
e_var_ss = True
t_var_ss = True
e_var_margins = False

topography_name = "ice6g_laea_5km_eurasia"
topography_nc = "/nfs/annie/cm15ogp/phd/data/ice_models/ice6g/ice6g_laea_5km_eurasia.nc"

shear_stress_map_name = "ely_victor_eurasia"
shear_stress_map_file = "/nfs/annie/cm15ogp/phd/data/shear_stress/eurasia/regions.shp"

In [ ]:
# topo0
t_var_topo = False
e_var_topo = False
e_var_ss = True
t_var_ss = True
e_var_margins = False

margins_name = "batchelor_mis6"
margins_dir = "/nfs/annie/cm15ogp/phd/data/ice_margins/batchelor_quickfix"
run_dir = "/nfs/annie/cm15ogp/phd/projects/2020-12-01_icesheet_calibration/experiments/shearstress_lhs_process/batchelor/"

topography_name = "rtopo2_laea_5km_eurasia"
topography_nc = "/nfs/annie/cm15ogp/phd/data/topography/rtopo2/rtopo2_laea_5km_eurasia.nc"


times = [-140000]
shear_stress_parameters_var_ranges = {
    "g_pgm_ice_streams_ice_stream": [5000,20000],
    "g_sediment_marine_sediment": [10000,30000],
    "g_sediment_onshore_sediment": [30000,100000],
    "g_sediment_bedrock": [100000,150000],
    "p_ice_stream_interior_dist": [0,1000],
    "p_cold_ice_shear_stress": [120000,200000],
    "p_cold_ice_interior_dist": [1000,300],
    "p_hybrid_ice_stream_shear_stress": [5000,100000],
    "p_hybrid_ice_stream_dist": [0,600],
}
shear_stress_parameters_const = {
    "m_base_map_name": shear_stress_map_name,
    "m_base_map_file": shear_stress_map_file,
    "m_gauss_blur": 1,
    "m_base_value": 5000,
    "m_combine_pattern": "((g_sediment | (g_pgm_ice_streams * p_ice_stream)) | p_hybrid_ice_stream) | p_cold_ice",
    "g_layers": ["sediment", "pgm_ice_streams"],
    "p_processes": ["ice_stream", "cold_ice", "hybrid_ice_stream"],
}

In [ ]:
# topo1
shear_stress_parameters_var_ranges = {
    "g_pgm_ice_streams_ice_stream": [5000,20000],
    "g_sediment_marine_sediment": [10000,30000],
    "g_sediment_onshore_sediment": [30000,100000],
    "g_sediment_bedrock": [100000,150000],
    "p_ice_stream_interior_dist": [0,1000],
    "p_cold_ice_shear_stress": [120000,200000],
    "p_cold_ice_interior_dist": [1000,300],
    "p_hybrid_ice_stream_shear_stress": [5000,100000],
    "p_hybrid_ice_stream_dist": [0,600],
}
run_dir = "/nfs/annie/cm15ogp/phd/paper1/paper1_data/icesheet_runs/batchelor/topo_iteration_1"
t_var_topo = True
e_var_topo = True
e_var_ss = True
t_var_ss = False
e_var_margins = False

times = [-140000]

In [ ]:
# glac corners
shear_stress_parameters_var_ranges = {
    "g_lgm_ice_streams_ice_stream": [5000,20000],
    "g_sediment_marine_sediment": [10000,30000],
    "g_sediment_onshore_sediment": [30000,100000],
    "g_sediment_bedrock": [100000,150000],
    "p_ice_stream_interior_dist": [0,1000],
    "p_cold_ice_shear_stress": [120000,200000],
    "p_cold_ice_interior_dist": [1000,300]
}
shear_stress_parameters_const = {
    "m_base_map_name": shear_stress_map_name,
    "m_base_map_file": shear_stress_map_file,
    "m_gauss_blur": 1,
    "m_base_value": 5000,
    "m_combine_pattern": "(g_sediment | (g_lgm_ice_streams * p_ice_stream)) | p_cold_ice",
    "g_layers": ["sediment", "lgm_ice_streams"],
    "p_processes": ["ice_stream", "cold_ice"],
}
times = [-22000,-20000,-18000,-16000]
margins_name = "glac1d_smooth_v1_topoclip_mod_-1km"
margins_dir = "/nfs/annie/cm15ogp/phd/data/ice_margins/glac1d/smooth_v1_topoclip_mod_-1km"
run_dir = "/nfs/annie/cm15ogp/phd/paper1/paper1_data/icesheet_runs/glac1d/corners/"

shear_stress_parameters = {'m_base_map_name': 'ely_victor_eurasia',
 'm_base_map_file': '/nfs/annie/cm15ogp/phd/data/shear_stress/eurasia/regions.shp',
 'm_gauss_blur': 0,
 'm_base_value': 5000,
 'm_combine_pattern': '(g_lgm_ice_streams * p_ice_stream)',
 'g_layers': ['sediment', 'lgm_ice_streams'],
 'p_processes': ['ice_stream', 'cold_ice'],
 'g_lgm_ice_streams_ice_stream': 20000,
 'g_sediment_marine_sediment': 30000,
 'g_sediment_onshore_sediment': 100000,
 'g_sediment_bedrock': 150000,
 'p_ice_stream_interior_dist': 0,
 'p_cold_ice_shear_stress': 200000,
 'p_cold_ice_interior_dist': 300,
 'p_margin_name': 'glac1d_smooth_v1_topoclip_mod_-1km',
 'p_margin_time': -16000,
 'p_margin_file': '/nfs/annie/cm15ogp/phd/data/ice_margins/glac1d/smooth_v1_topoclip_mod_-1km/-16000/margin.shp'}

topography_name = "glac1d_laea_5km_eurasia"
topography_nc = "/nfs/annie/cm15ogp/phd/data/ice_models/glac1d/glac1d_topo_laea_5km_eurasia.nc"
t_var_topo = True
e_var_topo = False
e_var_ss = True
t_var_ss = True
t_var_margins = True
e_var_margins = False

In [ ]:
grid_x = np.arange(x_min, x_max+dx, dx)
grid_y = np.arange(y_min, y_max+dy, dy)

# Parameter Sample

In [ ]:
shear_stress_parameters_const = {
    "m_base_map_name": shear_stress_map_name,
    "m_base_map_file": shear_stress_map_file,
    "m_gauss_blur": shear_stress_gauss_blur,
    "m_base_value": shear_stress_base_value,
    "m_combine_pattern": shear_stress_combine_pattern,
    "g_layers": shear_stress_sediment_layers,
    "p_processes": shear_stress_process_layers,
}

if Path(sample_norm_path).exists():
    print("Loading Previous Sample")
    sample = np.genfromtxt(sample_norm_path, delimiter=",", skip_header=1)
    sample_values = np.genfromtxt(sample_values_path, delimiter=",", skip_header=1)
else:
    print("Generating New Sample")
    if sample_type == "lhs":
        sample_norm = gen_lhs_from_dict(
            parameter_dict=shear_stress_parameters_var_ranges,
            n_samples=n_sample,
            save=lhs_norm_path
        )

        sample_values = norm_to_values(
            lhs_norm_values=sample_norm,
            parameter_ranges=list(shear_stress_parameters_var_ranges.values()),
            parameter_names=list(shear_stress_parameters_var_ranges.keys()),
            save=lhs_values_path
        )
        
    elif sample_type == "corners":
        ranges = np.array(list(shear_stress_parameters_var_ranges.values()))
        min_vals, max_vals = ranges[:,0], ranges[:,1]

        def vertices(N): 
            return np.array(list(product((0, 1), repeat=N)))
        
        sample_norm = vertices(7)
        sample_values = sample_norm * (max_vals - min_vals) + min_vals
    
exp_vars = pd.DataFrame(sample_values, columns=list(shear_stress_parameters_var_ranges.keys()))


# Generate Shear Stress Inputs

In [ ]:
for time in tqdm(times):
    margin_filename = str(Path(margins_dir) / str(time) / "margin.shp")
    shear_stress_parameters_margins = {
        "p_margin_name": margins_name,
        "p_margin_time": time,
        "p_margin_file": margin_filename,
    }
    for exp_name, values in tqdm(exp_vars.iterrows(), total=len(exp_vars)):
        inputs_dir_shear_stress = (
            Path(run_dir) / 
            'inputs' / 
            'shearstress' / 
            str(time) / 
            str(exp_name)
        )
        inputs_dir_shear_stress.mkdir(exist_ok=True, parents=True)
        shear_stress_parameters_var = values.to_dict()
        shear_stress_parameters = {
            **shear_stress_parameters_const, 
            **shear_stress_parameters_var, 
            **shear_stress_parameters_margins,
        }

        make_shear_stress_map(
            grid_x = grid_x, 
            grid_y = grid_y, 
            grid_crs = grid_crs, 
            parameters = shear_stress_parameters,
            save=str(inputs_dir_shear_stress / "shearstress.nc")
        )


# Generate Margin Inputs

In [ ]:
inputs_dir_margins = (
    Path(run_dir) / 
    'inputs' / 
    'margins'
)
inputs_dir_margins.mkdir(exist_ok=True, parents=True)

shp_series_to_icesheet(
    margins_dir, 
    grid_crs, 
    str(inputs_dir_margins), 
    times=times
)

# Generate Topography Inputs

In [ ]:
inputs_dir_topography = (Path(run_dir) / "inputs/topography")
inputs_dir_topography.mkdir(exist_ok=True, parents=True)
topography_to_icesheet_nc(topography_nc=topography_nc, output_dir=inputs_dir_topography)

# Generate Run Commands

In [ ]:
run_configs = []
for member, config in exp_vars.iterrows():
    config = config.to_dict()
    run_config = gen_run_configs(
        run_dir = str(run_dir),
        times = times,
        member_name = str(member),
        t_var_topo=t_var_topo,
        e_var_topo=e_var_topo,
        t_var_ss=t_var_ss,
        e_var_ss=e_var_ss,
        e_var_margins=e_var_margins
    )
    run_configs = run_configs + run_config

# Run ICESHEET Model

In [ ]:
ism = IcesheetModel(icesheet_model_binary)
ism.run_pool(run_configs, cores=num_run_cores)